Библиотеки

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

1.1 Определение функции 1

In [ ]:
def f1(x):
    return np.log10(x + 2) + x

1.2 Определение функции 2

In [ ]:
def f2(x):
    return np.power(x, 3) - 0.1 * np.power(x, 2) + 0.4 * np.abs(x) + 2

2.1 Создание равномерной сетки

In [ ]:
def uniform_grid(a, b, n):
    return np.linspace(a, b, n)

2.2 Создание Чебышевской сетки

In [ ]:
def chebyshev_grid(a, b, n):
    k = np.arange(n)
    nodes = np.cos((2 * k + 1) * np.pi / (2 * n))
    return 0.5 * (a + b) + 0.5 * (b - a) * nodes

3. Базисные полиномы Лагранжа

In [ ]:
def lagrange_basis(x, xt, k):
    return np.prod([(x - xt[i]) / (xt[k] - xt[i]) for i in range(len(xt)) if i != k], axis=0)

4. Интерполяционный полином Лагранжа

In [ ]:
def lagrange_polinom(x, xt, yt):
    return sum(yt[k] * lagrange_basis(x, xt, k) for k in range(len(xt)))

5. Вспомогательная функция для выбора сетки

In [ ]:
def select_grid(grid_type, a, b, n):
    if grid_type == 'uniform':
        return uniform_grid(a, b, n)
    elif grid_type == 'chebyshev':
        return chebyshev_grid(a, b, n)
    else:
        raise ValueError(f"Неверный тип сетки. Используйте 'uniform' или 'chebyshev'.")

6. Построение графиков интерполяции

In [ ]:
def plot_int(f, a, b, n_val, grid_type):
    x_plot = np.linspace(a, b, 1000)
    y_plot = f(x_plot)

    plt.figure(figsize=(21, 8))
    plt.plot(x_plot, y_plot, label='Исходная функция', color='black', linewidth=2)

    colors = ['blue', 'green', 'red']
    for i, n in enumerate(n_val):
        xt, yt = select_grid(grid_type, a, b, n), f(select_grid(grid_type, a, b, n))
        y_int = np.array([lagrange_polinom(x, xt, yt) for x in x_plot])

        plt.plot(x_plot, y_int, color=colors[i], linestyle='--', label=f'n = {n}')
        plt.scatter(xt, yt, color=colors[i])
    plt.legend()
    plt.title(f'Интерполяция полиномом Лагранжа ({grid_type} сетка)')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid()

7. Построение поточечной ошибки интерполяции и графика теоретической ошибки

In [ ]:
def plot_error(f, a, b, n_val, grid_type):
    if f == f1:
        x_plot = np.linspace(a, b, 1000)
    else:
        x_plot = np.linspace(a, b, 100)
    y_plot = f(x_plot)

    plt.figure(figsize=(21, 8))

    colors = ['blue', 'green', 'red']
    for i, n in enumerate(n_val):
        xt, yt = select_grid(grid_type, a, b, n), f(select_grid(grid_type, a, b, n))
        y_int = np.array([lagrange_polinom(x, xt, yt) for x in x_plot])
        error = np.abs(y_plot - y_int)

        plt.plot(x_plot, error, color=colors[i], linestyle='-', label=f'Ошибка при n = {n}')

    omega = np.prod([(x_plot - xi) for xi in xt], axis=0)
    error_theor = 0.0005 * np.abs(omega) / np.math.factorial(4 + 1)

    if f == f1:
        plt.plot(x_plot, error_theor, color='purple', linestyle='dotted', label=f'Теоретическая ошибка')
    plt.legend()
    plt.title(f'Ошибка интерполяции ({grid_type} сетка)')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid()

8. Построение зависимости ошибки от количества узлов

In [ ]:
def max_error(f, a, b, n, grid_type):
    x_plot = np.linspace(a, b, 1000)
    y_plot = f(x_plot)

    xt, yt = select_grid(grid_type, a, b, n), f(select_grid(grid_type, a, b, n))
    y_int = np.array([lagrange_polinom(x, xt, yt) for x in x_plot])
    return np.max(np.abs(y_plot - y_int))

def plot_error_nodes(f, a, b, n_range, grid_type='uniform'):
    errors = [max_error(f, a, b, n, grid_type) for n in n_range]

    plt.figure(figsize=(21, 8))
    plt.plot(n_range, errors, marker='o', linestyle='-', color='blue')
    plt.title(f'Зависимость ошибки от количества узлов ({grid_type} сетка)')
    plt.xlabel('Количество узлов')
    plt.ylabel('Максимальная ошибка')
    plt.yscale('log')
    plt.grid()

9. Построение графиков зависимости ошибки от количества узлов в выбранных точках

In [ ]:
def error_at_points(f, a, b, n_range, grid_type):
    x_plot = np.linspace(a, b, 1000)
    y_plot = f(x_plot)

    point1 = x_plot[100]
    point2 = x_plot[-100]

    errors_point1 = []
    errors_point2 = []

    for n in n_range:
        xt, yt = select_grid(grid_type, a, b, n), f(select_grid(grid_type, a, b, n))
        y_int1 = lagrange_polinom(np.array([point1]), xt, yt)
        y_int2 = lagrange_polinom(np.array([point2]), xt, yt)
        error1 = np.abs(f(point1) - y_int1[0])
        error2 = np.abs(f(point2) - y_int2[0])

        errors_point1.append(error1)
        errors_point2.append(error2)

    return point1, point2, errors_point1, errors_point2

def plot_errors_at_points(f, a, b, n_range, grid_type='uniform'):
    point1, point2, errors_point1, errors_point2 = error_at_points(f, a, b, n_range, grid_type)

    plt.figure(figsize=(21, 8))
    plt.subplot(1, 2, 1)
    plt.plot(n_range, errors_point1, marker='o', linestyle='-', color='blue')
    plt.title(f'Ошибка в точке х = {point1:.2f} ({grid_type} сетка)')
    plt.xlabel('Количество узлов')
    plt.ylabel('Ошибка')
    plt.yscale('log')
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.plot(n_range, errors_point2, marker='o', linestyle='-', color='red')
    plt.title(f'Ошибка в точке х = {point2:.2f} ({grid_type} сетка)')
    plt.xlabel('Количество узлов')
    plt.ylabel('Ошибка')
    plt.yscale('log')
    plt.grid()

    plt.tight_layout()

10. Проведение эксперимента с возмущением данных

In [ ]:
def relative_error(f, a, b, xt, yt_noisy):
    x_plot = np.linspace(a, b, 1000)
    y_plot = f(x_plot)
    y_int = np.array([lagrange_polinom(x, xt, yt_noisy) for x in x_plot])

    return np.max(np.abs(y_plot - y_int) / np.abs(y_plot + 1e-10))

def experiment_noise(f, a, b, n, perturb_levels, grid_type, num=20):
    xt, yt = select_grid(grid_type, a, b, n), f(select_grid(grid_type, a, b, n))
    results = {level: {'actual_perturb': [], 'errors': []} for level in perturb_levels}

    for level in perturb_levels:
        for _ in range(num):
            perturb = np.random.uniform(-level, level, size=len(yt))
            yt_noisy = yt * (1 + perturb)
            results[level]['actual_perturb'].append(np.max(np.abs(perturb)))
            results[level]['errors'].append(relative_error(f, a, b, xt, yt_noisy))

    return results

def plot_noise_error(results, perturb_levels):
    plt.figure(figsize=(21, 8))
    boxplot_data, x_ticks = [], []

    for level in perturb_levels:
        boxplot_data.append(results[level]['errors'])
        x_ticks.append(round(np.max(results[level]['actual_perturb']), 4))
    x_ticks = [round(l * 100) for l in x_ticks]

    plt.boxplot(boxplot_data, positions=x_ticks, widths=0.05, showfliers=False)
    plt.title('Зависимость ошибки интерполяции от возмущения данных')
    plt.xlabel('Максимальное фактическое возмущение данных, %')
    plt.ylabel('Относительная ошибка интерполяции, %')
    plt.xlim(-0.5, 5.5)
    plt.grid()

Основная программа

In [ ]:
a, b = 0, 10
n_val = [4, 7, 10]
n_range = range(5, 51)
perturb_levels = [0.01, 0.02, 0.03, 0.04, 0.05]

for f in [f1, f2]:
    for grid in ['uniform', 'chebyshev']:
        plot_int(f, a, b, n_val, grid)
        plot_error(f, a, b, n_val, grid)
        plot_error_nodes(f, a, b, n_range, grid)
        plot_errors_at_points(f, a, b, n_range, grid)

Дополнительное исследование

In [ ]:
results1 = experiment_noise(f1, a, b, 10, perturb_levels, grid_type='uniform')
results2 = experiment_noise(f1, a, b, 10, perturb_levels, grid_type='chebyshev')
results3 = experiment_noise(f2, a, b, 10, perturb_levels, grid_type='uniform')
results4 = experiment_noise(f2, a, b, 10, perturb_levels, grid_type='chebyshev')
plot_noise_error(results1, perturb_levels)
plot_noise_error(results2, perturb_levels)
plot_noise_error(results3, perturb_levels)
plot_noise_error(results4, perturb_levels)